### Trabalho 3 - Inversores
###### Grupo 19

Tiago Passos Rodrigues - A96414

### Enunciado


O seguinte sistema dinâmico denota 4 inversores ($\,A, B, C, D\,$) que lêm um bit num canal input e escrevem num canal output uma transformação desse bit.

   i. Cada inversor tem um bit $s$ de estado, inicializado  com um valor aleatório.
   
   ii. Cada inversor é regido pelas seguintes transformações
$$\mathbf{invert}\mathtt(in,out)$$
$$x \gets \mathsf{read}(\mathtt{in})$$
$$s \gets \neg x\;\;\|\;\; s\gets s\oplus x$$            
$$\mathsf{write}(\mathtt{out},s)$$
   iii. A escolha neste comando é sempre determinística; isto é, em cada inversor a escolha do comando a executar é sempre a mesma. Porém qual é essa escolha é determinada aleatoriamente na inicializarão do sistema.
   
   iii. O estado do sistema é um duplo definido pelos 4 bits $s$, e é inicializado com um vetor aleatório em $\{0,1\}^4\;$.
   
   iv. O sistema termina em ERRO quando o estado do sistema for $\,(0,0,0,0)\,$.
                    
   1. Construa um SFOTS que descreva este sistema e implemente este sistema, numa abordagem BMC (“bouded model checker”) num traço com $\,n\,$ estados.
   2. Verifique se o sistema é seguro usando BMC, k-indução ou model checking com interpolantes.

### Implementação

In [ ]:
from pysmt.shortcuts import *
from pysmt.typing import *



# função que cria a "próxima variável" no sistema de transição
def following(x):
    return Symbol(f'{x.symbol_name()}_', x.symbol_type())

# função que cria um clone de uma variável para um determinado step
def next_(x, step):
    return Symbol(f"{x.symbol_name()}{step}", x.symbol_type())

# classe que representa um FOTS
class FOTS:

    def __init__(self, label, variables, init, trans):
        """
        Construtor do FOTS
        
        Parâmetros:
            - label : indica o nome do conjunto das variáveis pertencentes a este FOTS.
                      De acordo com os apontamentos, seria equivalente a 'X'.
            
            - variables: uma lista que representa o conjunto das variáveis do FOTS.
                         Ao ser construído, o FOTS transforma todas as variáveis deste conjunto
                         num clone com a label atrás. Por exemplo, se passarmos  a label 'X' e o conjunto [z, pc],
                         o que o FOTS guarda é o conjunto [X_z, X_pc]
            
            - init: Predicado do estado inicial. Assume-se que usa um subconjunto das variáveis do FOTS.
                    O FOTS faz automaticamente a conversão das variáveis para a versão com label.

            - trans: Predicado da relação de transição. Assume-se que usa um subconjunto das variáveis do FOTS, e 
                     faz uso da função de following.
                     O FOTS faz automaticamente a conversão das variáveis para a versão com label.
        """

        self.label = label

        self.create_variables(variables)	
        self.create_init(variables, init)
        self.create_trans(variables, trans)


    def create_variables(self, variables):
        self.variables = [ Symbol(f'{self.label}_{v.symbol_name()}', v.symbol_type()) for v in variables ]

        self.original_variables = variables

    def create_init(self, variables, init):
        self.init = init.substitute( { variables[i]: self.variables[i] for i in range(len(variables)) } )

        self.original_init = init


    def create_trans(self, variables, trans):
        subs = { variables[i]: self.variables[i] for i in range(len(variables)) }
        for v in variables:
            v = following(v)
            subs[v] = Symbol(f'{self.label}_{v.symbol_name()}', v.symbol_type())

        self.trans = trans.substitute(subs)
        self.original_trans = trans


    # função que retorna um dicionário de substituições com os clones de um determinado estado
    def get_substitutes(self, i):
        # completar
        dic = {}
        for var in self.variables:
            dic[var] = next_(var,i)
            dic[following(var)] = next_(var,i+1)
        return dic
    
    #def get_substitutes2(self,i):
    #    return { v: next_(v,i) for v in self.variables} | {following(v): next_(v,i+1) for v in self.variables}
        
    def get_vars(self, i):
        # completar
        return [next_(v,i) for v in self.variables]

    # função que retorna o estado inicial do FOTS
    def get_init(self):
        # completar
        subs = self.get_substitutes(0)
        return self.init.substitute(subs)

    # função que retorna o predicado de transição num certo estado
    def get_trans(self, state):
        # completar
        subs = self.get_substitutes(state)
        return self.trans.substitute(subs)

    # função que returna a expansão do predicado de transição para k passos
    def expand_trans(self, k):
        # T(0,1) /\ ... /\ T(k-1, k)
        # completar
        return And([self.get_trans(i) for i in range(k)])

    # Funções para propriedades

    # função que recebe uma propriedade e muda as variáveis para a sua versão com label
    def parse_prop(self, prop):
        subs = { v: Symbol(f'{self.label}_{v.symbol_name()}', v.symbol_type()) for v in prop.get_free_variables() }

        return prop.substitute(subs)

    # função que retorna uma propriedade num certo estado
    def get_prop(self, prop, state):
        # completar
        subs = self.get_substitutes(state)
        return prop.substitute(subs)

    # função que expande uma propriedade para k passos
    def expand_prop(self, prop, k):
        # P(0) /\ P(1) /\ ... /\ P(k) 
        # completar
        return And([ self.get_prop(prop, i) for i in range(k+1) ])

    def check_property(self, prop, k):
        prop = self.parse_prop(prop)
        #print(prop)
        return self.k_induction(prop, k)


    def k_induction(self, prop, k):
        # completar
        I = self.get_init()
        Tk_1 = self.expand_trans(k-1)
        Pk_1 = self.expand_prop(prop,k-1)
        
        r = is_sat(And(I, Tk_1, Not(Pk_1)))
        #r = get_model(And(I, Tk_1, Not(Pk_1)))
        if r:
            #print(r)
            print("Property Invalid")
            return
            
        Pk = self.expand_prop(prop,k)
        Tk = self.expand_trans(k)
        Pk1 = self.expand_prop(prop,k+1)
        
        r = get_model(And(Pk,Tk, Not(Pk1)))
        if r:
            #print(r)
            print("Invalid Property")
            return
            
        print(f"Property is valid in the first {k} steps")


In [ ]:
class SFOTS(FOTS):

    def __init__(self, label, variables, init, trans, error):
        # completar
        super().__init__(label,variables,init,trans)
        self.create_error(variables, error)
        pass


    def create_error(self, variables, error):
        # completar
        pass


    def inverse_original_trans(self):
        # completar
        pass


    def dual(self, label):
        # completar
        pass

    def model_checking():
        pass # nos apontamentos teoricos


### Criação do SFOTS

In [ ]:
import numpy

pc = Symbol('pc',INT)
inv_a = Symbol('Inv_A', INT)
inv_b = Symbol('Inv_B', INT)
inv_c = Symbol('Inv_C', INT)
inv_d = Symbol('Inv_D', INT)

funcao = numpy.random.choice([0,1])

init = And(Equals(pc,Int(0)), 
          Equals(inv_a,numpy.random.choice(Int(0),Int(1))),
           Equals(inv_b,numpy.random.choice(Int(0),Int(1))),
           Equals(inv_c,numpy.random.choice(Int(0),Int(1))),
           Equals(inv_d,numpy.random.choice(Int(0),Int(1))),
           Not(And(Equals(inv_a,Int(0)),Equals(inv_b,Int(0)),Equals(inv_c,Int(0)),Equals(inv_d,Int(0)))
          )


trans = Or(
    And(Equals(pc,Int(0)), Equals(funcao,Int(0)), Equals(following(pc),Int(1)), Equals(inv_b,Not(inv_a)),
    And(Equals(pc,Int(0)), Equals(funcao,Int(1)), Equals(following(pc),Int(1)), Equals(inv_b,Xor(inv_a,inv_b)),
    And(Equals(pc,Int(1)), Equals(funcao,Int(0)), Equals(following(pc),Int(2)), Equals(inv_c,Not(inv_b)),
    And(Equals(pc,Int(1)), Equals(funcao,Int(1)), Equals(following(pc),Int(2)), Equals(inv_c,Xor(inv_b,inv_c)),
    And(Equals(pc,Int(2)), Equals(funcao,Int(0)), Equals(following(pc),Int(3)), Equals(inv_d,Not(inv_c)),
    And(Equals(pc,Int(2)), Equals(funcao,Int(1)), Equals(following(pc),Int(3)), Equals(inv_d,Xor(inv_c,inv_d)),
    And(Equals(pc,Int(3)), Equals(funcao,Int(0)), Equals(following(pc),Int(0)), Equals(inv_a,Not(inv_d)),
    And(Equals(pc,Int(3)), Equals(funcao,Int(1)), Equals(following(pc),Int(0)), Equals(inv_a,Xor(inv_d,inv_a)),
)
